# A first web scraping exercise

We're going to scrape a bunch of separate lists from one Wikipedia page. Wikipedia is a good place to practice scraping because the HTML there is well formatted, and the site's traffic is so high, they don't mind us hitting one page again and again. 

First we import two libraries: [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is for scraping, and [Requests](http://docs.python-requests.org/en/master/user/quickstart/) is for making the HTTP request to the server where we want to scrape.

**If you have not yet installed these two libraries** into the virtual environment where you are running this Notebook, you'll need to do that *first.*

In [1]:
from bs4 import BeautifulSoup
import requests

Most scraping scripts begin with one page, which means one URL, to be scraped. In the three lines below, the only thing that changes if you scrape a different page, even at another website, is the URL inside the single quotaton marks.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Florida'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

As a result of those three lines above, we now have a BeautifulSoup "object" named `soup`, and it contains **all the HTML** from the file at the URL we gave. THe HTML is stored in a manner that allows the BeautifulSoup functions to search and use the data.

Our goal is to scrape **all the lists** of colleges from that one Wikipedia page and put them in a CSV file, with separate columns for college name, location, type of college, and the URL for that college's Wikipedia page.

We start by using Chrome Dev Tools' "Inspect": right-click on the heading "State University System" and select "Inspect" from the pop-up menu.

If we inspect the heading above each of the lists of colleges and schools, we find that all of those headings are H3 headings. 

In [3]:
# start by collecting all the H3 elements on the page into a Python list
h3_list = soup.find_all('h3')

In [4]:
# we don't really NEED to print them, but we do it so we can see what we got
print(h3_list)

[<h3><span class="mw-headline" id="State_University_System">State University System</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_colleges_and_universities_in_Florida&amp;action=edit&amp;section=2" title="Edit section: State University System">edit</a><span class="mw-editsection-bracket">]</span></span></h3>, <h3><span class="mw-headline" id="Florida_College_System">Florida College System</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_colleges_and_universities_in_Florida&amp;action=edit&amp;section=3" title="Edit section: Florida College System">edit</a><span class="mw-editsection-bracket">]</span></span></h3>, <h3><span class="mw-headline" id="Other_public_institutions">Other public institutions</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_colleges_and_universities_in_Florida&am

Look closely, and you'll see that all the H3 heading elements are items in a Python *list.*

In [5]:
# the heading above the first list of schools we want is 'State University System'
# so let's get the first UL that comes after that heading
the_ul = ""
for head in h3_list:
    if head.span.get_text() == 'State University System':
        the_ul = head.find_next('ul')
        break
print(the_ul)

<ul><li><a href="/wiki/Florida_A%26M_University" title="Florida A&amp;M University">Florida A&amp;M University</a> (<a href="/wiki/Tallahassee,_Florida" title="Tallahassee, Florida">Tallahassee</a>)</li>
<li><a href="/wiki/Florida_Atlantic_University" title="Florida Atlantic University">Florida Atlantic University</a> (<a href="/wiki/Boca_Raton,_Florida" title="Boca Raton, Florida">Boca Raton</a>)</li>
<li><a href="/wiki/Florida_Gulf_Coast_University" title="Florida Gulf Coast University">Florida Gulf Coast University</a> (<a href="/wiki/Fort_Myers,_Florida" title="Fort Myers, Florida">Fort Myers</a>)</li>
<li><a href="/wiki/Florida_International_University" title="Florida International University">Florida International University</a> (<a href="/wiki/Miami" title="Miami">Miami</a>)</li>
<li><a href="/wiki/Florida_Polytechnic_University" title="Florida Polytechnic University">Florida Polytechnic University</a> (<a href="/wiki/Lakeland,_Florida" title="Lakeland, Florida">Lakeland</a>)</l

Again, printing wasn't strictly necessary, but it's good to make sure you got what you were trying to get. Otherwise, if you start getting error messages, you won't understand why. You should be able to recognize that you got the complete UL element and everything it contains. It is NOT in a list. It is one single BeautifulSoup tag object, and it's been assigned to the variable `the_ul`.

Above, you began with a *list* of all H3 elements on the page. Each H3 element from a Wikipedia page contains `span` tags and text and more. (You saw this with "Inspect.")

The next task was to *get* the complete UL element that comes after the heading we chose: "State University System." So we *looped* over the list of all H3 elements, and we checked *each one* to see if the text inside its *first* `span` tag matched the string we provided.

As soon as we found that exact string, we stopped, and we grabbed *the next UL element* that comes after that H3, and we assigned that entire UL element to the variable `the_ul`.

`break` makes the for-loop stop looping.

In [6]:
# next, we want to get 3 things out of each LI element in that list: 
# college name, location, and the URL for that college's Wikipedia page 
# use Chrome Dev Tools' "Inspect" again to see the structure of the HTML and figure out how to use it

schools = the_ul.find_all('li')

for li in schools:
    a_list = li.find_all('a')
    college_name = a_list[0].get_text()
    location = a_list[1].get_text()
    url = a_list[0]['href']
    print(college_name, location, url)


Florida A&M University Tallahassee /wiki/Florida_A%26M_University
Florida Atlantic University Boca Raton /wiki/Florida_Atlantic_University
Florida Gulf Coast University Fort Myers /wiki/Florida_Gulf_Coast_University
Florida International University Miami /wiki/Florida_International_University
Florida Polytechnic University Lakeland /wiki/Florida_Polytechnic_University
Florida State University Tallahassee /wiki/Florida_State_University
New College of Florida Sarasota /wiki/New_College_of_Florida
University of Central Florida Orlando /wiki/University_of_Central_Florida
University of Florida Gainesville /wiki/University_of_Florida
University of North Florida Jacksonville /wiki/University_of_North_Florida
University of South Florida Tampa /wiki/University_of_South_Florida
University of West Florida Pensacola /wiki/University_of_West_Florida


Line 5 above - `schools = the_ul.find_all('li')` - makes a Python *list* of all the LI elements in the URL we selected. Once we have a list, we can *loop over it* and do things with each element, separately and individually. In this loop, `li` is the variable we made to represent each individual LI element in the list, which is named `schools`.

One by one, for one LI at a time, we:

* Find all the A elements in it and put them into a new list.
* From the *first* A element, `a_list[0]`, we get the text inside the A tags and assign it to the variable `college_name`.
* From the *second* A element, `a_list[1]`, we get the text inside the A tags and assign it to the variable `location`.
* From the *first* A element, `a_list[0]`, we get the *value* of the `href` attribute and assign it to the variable `url`.
* We print the three variables for each time the loop loops.

In [7]:
# that's only a partial URL, but it will work if we concatenate it with 'https://en.wikipedia.org'
# let's do it over -
for li in schools:
    a_list = li.find_all('a')
    college_name = a_list[0].get_text()
    location = a_list[1].get_text()
    url = 'https://en.wikipedia.org' + a_list[0]['href']
    print(college_name, location, url)


Florida A&M University Tallahassee https://en.wikipedia.org/wiki/Florida_A%26M_University
Florida Atlantic University Boca Raton https://en.wikipedia.org/wiki/Florida_Atlantic_University
Florida Gulf Coast University Fort Myers https://en.wikipedia.org/wiki/Florida_Gulf_Coast_University
Florida International University Miami https://en.wikipedia.org/wiki/Florida_International_University
Florida Polytechnic University Lakeland https://en.wikipedia.org/wiki/Florida_Polytechnic_University
Florida State University Tallahassee https://en.wikipedia.org/wiki/Florida_State_University
New College of Florida Sarasota https://en.wikipedia.org/wiki/New_College_of_Florida
University of Central Florida Orlando https://en.wikipedia.org/wiki/University_of_Central_Florida
University of Florida Gainesville https://en.wikipedia.org/wiki/University_of_Florida
University of North Florida Jacksonville https://en.wikipedia.org/wiki/University_of_North_Florida
University of South Florida Tampa https://en.wiki

In [8]:
# now how can we put them into a CSV file? Python has a built-in module named csv -
# we have to import it to use it

import csv

# create/open new file for writing -
csvfile = open("test.csv", 'w', newline='', encoding='utf-8')

# make a new variable, c, for Python's CSV writer object -
c = csv.writer(csvfile)

# write header row to test.csv
c.writerow( ['first', 'second', 'third', 'fourth'] )

# write two more rows to test.csv
c.writerow( ['a', 'b', 'c', 'd'] )
c.writerow( [10, 20, 30, 40] )


# close and save the file
csvfile.close()


You have now created a new CSV file and written three rows and four columns into it. But WHERE was it *saved*? To find out, use the `pwd` to find out which directory this Jupyter Notebook is running in. (`pwd` is a command that stands for "print working directory.")

In [9]:
pwd

'/Users/mcadams/Documents/python/conda_jup/conda-python-intro/complete'

Go there now in your Finder/File Explorer and open the CSV file named *test.csv.*

Now you know how to write a new CSV file. Can you figure out how to take the earlier code from above - the for-loop that PRINTS a list of schools - and make it WRITE to a row in the CSV *instead* of printing?

Because that's all you need to do.

In [10]:
# create/open new file for writing -
csvfile = open("state_university_system.csv", 'w', newline='', encoding='utf-8')

# make a new variable, c, for Python's CSV writer object -
c = csv.writer(csvfile)

# write header row to the csv
c.writerow( ['school', 'location', 'url'] )

# the code from above that loops over all the schools in the list of LI elements
for li in schools:
    a_list = li.find_all('a')
    college_name = a_list[0].get_text()
    location = a_list[1].get_text()
    url = 'https://en.wikipedia.org' + a_list[0]['href']
    c.writerow( [college_name, location, url] )

# close and save the file
csvfile.close()


Open your new CSV, named *state_university_system.csv,* and take a look at the rows and columns.

An **important point** to understand about the new line in the code above - `c.writerow( [college_name, location, url] )` - is that the square brackets *inside* the parentheses are **expected** by the csv module's code. It expects to receive a Python list (which is like an array in JavaScript), and if it doesn't, the CSV file will not be written correctly.

So any time you WRITE a ROW to a CSV file here, you must write a *list* and NOT just variables separated by commas (which will not work).

## We said we would get all the schools from all the lists, and we will

What we've done is nice, but it will be even nicer when we get all the lists of schools that are still in operation (we will skip the closed schools). And we already have almost all the Python and BeautifulSoup code we need to get the job done.

If we are short on time for this demonstration, we will stop here.

Recall that we got a list of all the H3 headings. It's in the variable `h3_list`.

In [11]:
print(h3_list)

[<h3><span class="mw-headline" id="State_University_System">State University System</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_colleges_and_universities_in_Florida&amp;action=edit&amp;section=2" title="Edit section: State University System">edit</a><span class="mw-editsection-bracket">]</span></span></h3>, <h3><span class="mw-headline" id="Florida_College_System">Florida College System</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_colleges_and_universities_in_Florida&amp;action=edit&amp;section=3" title="Edit section: Florida College System">edit</a><span class="mw-editsection-bracket">]</span></span></h3>, <h3><span class="mw-headline" id="Other_public_institutions">Other public institutions</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_colleges_and_universities_in_Florida&am

That is all the complete H3 elements. Let's look at just the text of the headings instead.

In [12]:
for head in h3_list:
    if head.span:
        print(head.span.get_text())

State University System
Florida College System
Other public institutions

Religiously affiliated institutions

Other private institutions
Segregated junior colleges
Variants
More


I check the Wikipedia page and discover I am missing "Trade/technical institutions." I use "Inspect" again and see that the H3 element is a bit different from the others, so I try another way.

In [13]:
for head in h3_list:
    if head.find( 'span', {'class':'mw-headline'} ):
        print( head.find( 'span', {'class':'mw-headline'} ).get_text() )


State University System
Florida College System
Other public institutions
Defunct public (county) colleges in Florida
Religiously affiliated institutions
Trade/technical institutions
Other private institutions
Segregated junior colleges


The BeautifulSoup pattern `object.find( 'element', {'class':'classname'} )` is very, very common and very, very useful. 

`.find()` brings you only the first of the class, even if there are others on the page. 

`.find_all()` brings you a Python list of ALL the elements of that kind on the page that have that class.

`.get_text()` strips the text only out of the element and gives it to you.

When I examined the Wikipedia HTML carefully with "Inspect," I realized that all the headings are inside a `span` tag that has `class="mw-headline` - *so that is what I used* to get the printed H3 headlines I want.

Use Command-f (or Control-f on Windows) on the [BeautifulSoup documentation page](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to discover these tricks.

In [14]:
# copy/paste that nice text to create a new list 
# we will use this list of headings to grab all the schools we want 

headings = [
    'State University System',
    'Florida College System',
    'Other public institutions',
    'Religiously affiliated institutions',
    'Trade/technical institutions',
    'Other private institutions'
]

# I removed 'Defunct public (county) colleges in Florida' and 
#'Segregated junior colleges' because those are all closed 

I've checked the list carefully against the Wikipedia page. Every **list of schools** that I want to add to my CSV comes under one of those headings. I will use the headings the same way I used 'State University System' alone, earlier, to get ONE list of schools.

I will use the `h3_list` as I did before - but now I will ALSO use this new `headings` list.

In [15]:
for item in headings:
    the_ul = ""
    for head in h3_list:
        text = head.find( 'span', {'class':'mw-headline'} ).get_text()
        if text == item:
            the_ul = head.find_next('ul')
            break
    # only the first LI in the UL 
    print( the_ul.li.get_text() )

Florida A&M University (Tallahassee)
Broward College (Davie)
George Stone Technical Center (Pensacola)
Adventist University of Health Sciences (Orlando)
Acupuncture and Massage College (Miami)
By state and in insular areas


I printed only the text in the first LI in each list to test my code, and I have a problem. "By state and in insular areas" is not a school. Back on the Wikipedia page, I inspet again, and I find that the heading "Other private institutions" is followed (in the HTML) by a table, and *inside the table* there is a UL I do not want.

This is a typical scraping problem, and we can solve it.

In [16]:
for item in headings:
    the_ul = ""
    for head in h3_list:
        text = head.find( 'span', {'class':'mw-headline'} ).get_text()
        if text == item:
            the_ul = head.find_next('ul')
            # eliminate the ULs inside the table
            if the_ul.parent.name == 'td':
                the_ul = head.find_next('ul').find_next('ul').find_next('ul')
            break
    print( the_ul.li.get_text() )

Florida A&M University (Tallahassee)
Broward College (Davie)
George Stone Technical Center (Pensacola)
Adventist University of Health Sciences (Orlando)
Acupuncture and Massage College (Miami)
Argosy University (Tampa)


It's a clunky solution, but it works - as shown by the line "Argosy University (Tampa)," which is the first LI in that last list we want to grab. By the way, the trick with  `the_ul.parent.name` is something else I found on the [BeautifulSoup documentation page](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

`head.find_next('ul').find_next('ul').find_next('ul')` means not the first, not the second, get the third.

Since we now know we can get all the LI elements, let's try to add in the code that gets the location and the URL too.

In [17]:
for item in headings:
    the_ul = ""
    for head in h3_list:
        text = head.find( 'span', {'class':'mw-headline'} ).get_text()
        if text == item:
            the_ul = head.find_next('ul')
            # eliminate the ULs inside the table
            if the_ul.parent.name == 'td':
                the_ul = head.find_next('ul').find_next('ul').find_next('ul')
            break
    # print( the_ul.li.get_text() )
    
    schools = the_ul.find_all('li')

    for li in schools:
        a_list = li.find_all('a')
        college_name = a_list[0].get_text()
        location = a_list[1].get_text()
        url = 'https://en.wikipedia.org' + a_list[0]['href']
        print(college_name, location, url, item)


Florida A&M University Tallahassee https://en.wikipedia.org/wiki/Florida_A%26M_University State University System
Florida Atlantic University Boca Raton https://en.wikipedia.org/wiki/Florida_Atlantic_University State University System
Florida Gulf Coast University Fort Myers https://en.wikipedia.org/wiki/Florida_Gulf_Coast_University State University System
Florida International University Miami https://en.wikipedia.org/wiki/Florida_International_University State University System
Florida Polytechnic University Lakeland https://en.wikipedia.org/wiki/Florida_Polytechnic_University State University System
Florida State University Tallahassee https://en.wikipedia.org/wiki/Florida_State_University State University System
New College of Florida Sarasota https://en.wikipedia.org/wiki/New_College_of_Florida State University System
University of Central Florida Orlando https://en.wikipedia.org/wiki/University_of_Central_Florida State University System
University of Florida Gainesville https://

Western Michigan University Cooley Law School Tampa https://en.wikipedia.org/wiki/Western_Michigan_University_Cooley_Law_School Other private institutions


That looks pretty good, so let's insert that whole chunk of code *into* the CSV-writing code and give it a try.

In [18]:
# create/open new file for writing -
csvfile = open("florida_colleges.csv", 'w', newline='', encoding='utf-8')

# make a new variable, c, for Python's CSV writer object -
c = csv.writer(csvfile)

# write header row to test.csv
c.writerow( ['school', 'location', 'url', 'type'] )

# write all the schools into rows
for item in headings:
    the_ul = ""
    for head in h3_list:
        text = head.find( 'span', {'class':'mw-headline'} ).get_text()
        if text == item:
            the_ul = head.find_next('ul')
            # eliminate the ULs inside the table
            if the_ul.parent.name == 'td':
                the_ul = head.find_next('ul').find_next('ul').find_next('ul')
            break
    
    schools = the_ul.find_all('li')

    for li in schools:
        a_list = li.find_all('a')
        college_name = a_list[0].get_text()
        location = a_list[1].get_text()
        url = 'https://en.wikipedia.org' + a_list[0]['href']
        
        c.writerow( [college_name, location, url, item] )


# close and save the file
csvfile.close()


And it works! You have a clean CSV with 120+ schools, and you could use it to make a map, a database, a mailing list (that would require more scraping on other pages), etc. You can sort the schools alphabetically or by location and still know which type of schools they are because of the "type" column. Some schools might not be real (it is Wikipedia, after all), so some more human intelligence would need to be applied to the CSV before it could be used with confidence.

**Note** that *only the code in the final cell* (plus all the `import` statements from higher up) is needed to make this CSV. All the cells above that were testing and working to get to this point.